In [ ]:
import os
import jsonref
from dotenv import dotenv_values
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import OpenApiTool, OpenApiAnonymousAuthDetails
from azure.identity import DefaultAzureCredential

In [ ]:
env = dotenv_values(".env")

Note... before you run the cell below, be sure you've deployed the API code to a public endpoint, and update the `servers` section in [api.json](../api/api.json) with the endpoint URL. This is how the agent knows where to send requests.

In [ ]:
with open('../api/api.json', 'r') as f:
    openapi_spec = jsonref.loads(f.read())

auth = OpenApiAnonymousAuthDetails()

openapi = OpenApiTool(name="weather_tool", spec=openapi_spec, description="Retrieve weather information for a location", auth=auth)

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=env["FOUNDRY_CONN"]
)

agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="weather-agent",
    instructions="You are a weather bot. Use the provided functions to help answer questions.",
    tools=openapi.definitions
)

In [ ]:
thread = project_client.agents.create_thread()

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Do I need an umbrella in New York today?",
)


In [ ]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")